In [1]:
import findspark, random
findspark.init("/u/cs451/packages/spark")

from pyspark import SparkContext, SparkConf
sc = SparkContext(appName="YourTest", master="local[2]", conf=SparkConf().set('spark.ui.port', random.randrange(4000,5000)))

In [127]:
from simple_tokenize import simple_tokenize
from spamminess import spamminess
from math import exp
from operator import add
n=5
m=1000

In [35]:
dataset='bank_final.txt'

In [90]:
def generate_ngrams(sent):
    ngrams_list = []
    words_list = simple_tokenize(sent[0])
    for num in range(0, len(words_list)-n+1):
        ngram = ' '.join(words_list[num:num + n])
        ngrams_list.append(ngram)
 
    return (ngrams_list,sent[1])

In [ ]:
def tonums(interm):
    numlst = []
    strlst = interm[0]
    for ele in strlst:
        val = extractNGram.get(ele)
        numlst.append(val)
    return (numlst, interm[1])

In [149]:
def trainer(tngrm, n_itr, alpha, delta):
    w={}
    for itr in range(0,n_itr):
        d_w = toNGram.map(lambda x: (x[0], 1 if x[1]>0 else 0))\
            .map(lambda x: (x[0],x[1],1/(1+exp(-spamminess(x[0],w)))))\
            .map(lambda x: (x[0],(x[1]-x[2])*alpha))\
            .flatMap(lambda x: map(lambda y: (y,x[1]), x[0]))\
            .reduceByKey(add)
        w = d_w.map(lambda x: (x[0],x[1]+w.get(x[0],0))).collectAsMap()
        s_d_w =  d_w.map(lambda x: ("",abs(x[1]))).reduceByKey(add).take(1)
        if (s_d_w[0])[1] < delta:
            break
    return w

In [150]:
trainer(toNGram, 20,0.1,0.1)

{0: 0.13468503275507715,
 244: 0.13468503275507715,
 246: 0.13468503275507715,
 2: 0.13468503275507715,
 248: 0.13468503275507715,
 4: 0.13468503275507715,
 250: 0.13468503275507715,
 6: 0.13468503275507715,
 252: 0.13468503275507715,
 254: 0.13468503275507715,
 8: 0.13468503275507715,
 256: 0.13468503275507715,
 10: 0.13468503275507715,
 12: 0.13468503275507715,
 14: 0.13468503275507715,
 16: 0.21646329056819688,
 18: 0.21646329056819688,
 258: 0.21646329056819688,
 20: 0.21646329056819688,
 260: 0.21646329056819688,
 262: 0.21646329056819688,
 22: 0.21646329056819688,
 264: 0.21646329056819688,
 24: -0.16754869510135245,
 266: -0.16754869510135245,
 268: -0.16754869510135245,
 26: -0.16754869510135245,
 270: -0.16754869510135245,
 272: -0.16754869510135245,
 28: -0.16754869510135245,
 274: -0.16754869510135245,
 30: -0.16754869510135245,
 32: -0.16754869510135245,
 34: -0.16754869510135245,
 276: -0.16754869510135245,
 36: 0.15773177785120537,
 38: 0.15773177785120537,
 40: 0.1577317

In [77]:
sents = sc.textFile(dataset).map(lambda x: (x[:-3],x[-1:]))\
.map(lambda x: (x[0],1 if x[1]=="+" else -1 if x[1]=='-' else 0))

In [ ]:
extractNGram = sents\
.filter(lambda x: x[1]!=0).map(generate_ngrams).flatMap(lambda x: x[0])\
.distinct().zipWithIndex().filter(lambda x: x[1]<m)\
.collectAsMap()

In [91]:
toNGram = sents.filter(lambda x: x[1]!=0).map(generate_ngrams).map(tonums)

In [100]:
w={}
alpha =0.1

In [141]:
d_w = toNGram.map(lambda x: (x[0], 1 if x[1]>0 else 0))\
.map(lambda x: (x[0],x[1],1/(1+exp(-spamminess(x[0],w)))))\
.map(lambda x: (x[0],(x[1]-x[2])*alpha))\
.flatMap(lambda x: map(lambda y: (y,x[1]), x[0]))\
.reduceByKey(add)

In [135]:
d=d_w.collectAsMap()

In [147]:
s_d_w =  d_w.map(lambda x: ("",abs(x[1]))).reduceByKey(add).take(1)

In [148]:
(s_d_w[0])[1]

25.30000000000009